In [1]:
# from lrgsglib.Ccore.statsys.RBIsingM.base import ising_model
from lrgsglib.shared import *
from lrgsglib.core import *
from lrgsglib.Ccore.statsys.RBIsingM.storer import ising_model_store

In [2]:
move_to_rootf()

Current working directory: /home/opisthofulax/PhD/projects/complex-networks/lrgsglib


In [ ]:
def assign_random_signs(edge_list, p_flip):
    rand_p = np.random.rand(len(edge_list))
    signs = [1 if p_flip < rand_p[i] else -1 for i in range(len(edge_list))]
    return signs

In [ ]:
temperature = 1
pflip = 0
side = 50
navg = 5
steps = side**2
frameRate = 1
#
l = Lattice2D(side, geo='squared', pbc=True)
#
edgeList = list(l.GraphReprDict['H'].edges())
nodeList = l.nodeList
signs = assign_random_signs(edgeList, pflip)
# #
# model = ising_model_store.IsingModel(edgeList, signs, side, side, 
#                                         temperature, "asynchronous")
# model.simulate(steps, frameRate)

In [ ]:
magn = []
ene = []
for _ in range(navg):
    model = ising_model_store.IsingModel(edgeList, signs, side, side, 
                                         temperature, "asynchronous")
    model.simulate(steps, frameRate)
    magn.append([model.getFrameMagnetizations()])
    ene.append([model.getFrameEnergies()])

In [ ]:
for en in ene:
    tt = en[0]
    eneNorm = tt
    plt.plot(np.linspace(1, len(eneNorm), num=len(eneNorm)), eneNorm, '-')
# plt.plot(magn, 'h')
plt.xscale('log')


In [3]:
side = 50
pflip = 0.
T_ising = 1
IMPORT = False
if IMPORT:
    args = {'import_on': True}
else:
    args = {}

sqlatt = Lattice2D(
    side1=side,
    geo="squared",
    pflip=pflip,
    **args
)

ising_dyn = IsingDynamics(
    sg=sqlatt, 
    T=T_ising, 
    nstepsIsing=1000, 
    ic="uniform",
    runlang="C0", 
    # NoClust = 5
)
if not sqlatt.import_on:
    sqlatt.flip_random_fract_edges()
    sqlatt.export_graph()
    sqlatt.export_adj_bin()


In [4]:
energies = []
for i in range(10):
    ising_dyn.init_ising_dynamics()
    ising_dyn.run()
    sfxstr = ising_dyn.id_string_isingdyn if ising_dyn.id_string_isingdyn else "\"\""
    enepath = f"{sqlatt.isingpath}ene_{sqlatt.stdFname}_T={ising_dyn.T:.3g}_{sfxstr}.bin"
    ene = np.fromfile(enepath, dtype=float)
    energies.append(ene)

In [ ]:
for ene in energies:
    x = np.linspace(1, len(ene), num=len(ene))
    plt.plot(x, ene, '-')
plt.xscale('log')

In [ ]:
energies

In [ ]:
for i in range(10):
    if np.abs(magn[i][-1]) != 1.:
        plt.imshow(np.array(model_asynchronous.getFrameSpins()[-1]).reshape(side, side))
        plt.colorbar()
        plt.show()

In [ ]:
magn = model_asynchronous.getFrameMagnetizations()
plt.plot(np.linspace(1, len(magn),  num=len(magn)), np.array(magn), '-')

In [ ]:
width = 100
height = 100
temperature = 0
steps = 1000
frameRate = 100

edge_list, sign_list = create_edge_list_and_signs(width, height)
edge_list = [(int(edge[0]), int(edge[1])) for edge in edge_list]

# Initialize Ising model in sequential mode
# model_sequential = ising_model.IsingModel(edge_list, sign_list, width, height, temperature, "sequential")
# spins_sequential = model_sequential.getSpins()
# print("Spins before sequential simulation:")
# print(list(spins_sequential))
# model_sequential.simulate(steps)
# spins_sequential = model_sequential.getSpins()
# print("Spins after sequential simulation:")
# print(list(spins_sequential))

# Initialize Ising model in asynchronous mode
model_asynchronous = ising_model_store.IsingModel(edge_list, sign_list, width, height, temperature, "synchronous")
spins_asynchronous = model_asynchronous.getSpins()
print("Spins after asynchronous simulation:")
print(list(spins_asynchronous))
model_asynchronous.simulate(steps, frameRate)
spins_asynchronous = model_asynchronous.getSpins()
print("Spins after asynchronous simulation:")
print(list(spins_asynchronous))


In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
plt.imshow(np.array(model_asynchronous.getFrameSpins()[-1]).reshape(width, height), cmap='gray', interpolation='nearest')

In [ ]:
ene = model_asynchronous.getFrameEnergies()
plt.plot(np.linspace(1, len(ene),  num=len(ene)), np.array(ene), 'o')


In [ ]:
import matplotlib.pyplot as plt 

# plt.hist(list(spins_sequential))
plt.hist(list(spins_asynchronous))